In [0]:
import mlflow
import mlflow.spark
from mlflow.exceptions import RestException
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS

In [0]:
interactions = spark.table("MLOps.data.als_interactions_30d")

product_indexer = StringIndexer(
    inputCol="product_id",
    outputCol="product_id_idx",
    handleInvalid="skip"
)
interactions_indexed = product_indexer.fit(interactions).transform(interactions)

# als = ALS(
#     userCol="customer_id",
#     itemCol="product_id_idx",
#     ratingCol="interaction_weight",
#     implicitPrefs=True,
#     coldStartStrategy="drop",
#     rank=20,
#     maxIter=10,
#     regParam=0.1
# )

# als = ALS(
#     userCol="customer_id",
#     itemCol="product_id_idx",
#     ratingCol="interaction_weight",
#     implicitPrefs=True,
#     coldStartStrategy="drop",
#     rank=40,
#     maxIter=15,
#     regParam=0.05
# )

als = ALS(
    userCol="customer_id",
    itemCol="product_id_idx",
    ratingCol="interaction_weight",
    implicitPrefs=True,
    coldStartStrategy="drop",
    rank=10,
    maxIter=15,
    regParam=0.05
)

experiment_path = "/Workspace/Users/jung@ap-com.co.jp/mlops_demo_model/als_recommendation"

try:
    mlflow.set_experiment(experiment_path)
    print(f"Experiment found or created at: {experiment_path}")

except RestException as e:
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        experiment_id = mlflow.create_experiment(experiment_path)
        mlflow.set_experiment(experiment_path)
        print(f"Experiment created at: {experiment_path}, ID: {experiment_id}")
    else:
        raise e

with mlflow.start_run(run_name="training") as run:
    model = als.fit(interactions_indexed)
    mlflow.spark.log_model(model, "als_model")
    mlflow.log_params({
        "rank": 20,
        "maxIter": 10,
        "regParam": 0.1
    })
    run_id = run.info.run_id

print(f"Training finished. Run ID: {run_id}")

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
import mlflow

interactions = spark.table("MLOps.data.als_interactions_30d")

product_indexer = StringIndexer(
    inputCol="product_id",
    outputCol="product_id_idx",
    handleInvalid="skip"
)
interactions_indexed = product_indexer.fit(interactions).transform(interactions)

train, test = interactions_indexed.randomSplit([0.8, 0.2], seed=42)

experiment_path = "/Workspace/Users/jung@ap-com.co.jp/mlops_demo_model/als_recommendation"
experiment = mlflow.get_experiment_by_name(experiment_path)

runs_df = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id]
)
training_runs = runs_df[runs_df["tags.mlflow.runName"] == "training"]
latest_training_run = training_runs.sort_values(
    by="start_time", ascending=False
).iloc[0]

model_uri = f"runs:/{latest_training_run.run_id}/als_model"
model = mlflow.spark.load_model(model_uri)
print(model_uri)

predictions = model.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="interaction_weight",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)

with mlflow.start_run(run_name="evaluation") as run:
    mlflow.log_metric("rmse", rmse)
    print(f"Evaluation finished. RMSE: {rmse}")

In [0]:
import mlflow
from pyspark.ml.evaluation import RegressionEvaluator

model_name = "als_recommendation_model"

challenger = mlflow.spark.load_model(model_uri)

try:
    champion = mlflow.spark.load_model(f"models:/{model_name}/Production")
    print("Champion model exists in Production")
except Exception as e:
    champion = None
    print("No Champion model found. Challenger will be promoted directly.", e)

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="interaction_weight",  
    predictionCol="prediction"
)

predictions_challenger = challenger.transform(test)
challenger_rmse = evaluator.evaluate(predictions_challenger)
print("Challenger RMSE:", challenger_rmse)

if champion is not None:
    predictions_champion = champion.transform(test)
    champion_rmse = evaluator.evaluate(predictions_champion)
    print("Champion RMSE:", champion_rmse)
else:
    champion_rmse = None

if champion is None or challenger_rmse < champion_rmse:
    print("Challenger outperforms Champion (or no Champion exists). Promoting...")

    mlflow.register_model(model_uri, model_name)
    client = mlflow.tracking.MlflowClient()

    latest_version = client.get_latest_versions(model_name, stages=["None"])[-1].version

    client.transition_model_version_stage(
        name=model_name,
        version=latest_version,
        stage="Production",
        archive_existing_versions=True
    )
    print("Challenger promoted to Champion (Production).")
else:
    print("Challenger did not outperform Champion. No promotion.")


In [0]:
## 1回
Champion model exists in Production
Challenger RMSE: 0.9687680445911088
Champion RMSE: 0.9687680445911088
Challenger did not outperform Champion. No promotion.

## 2回
Champion model exists in Production
Challenger RMSE: 0.8180663645429364
Champion RMSE: 0.9687680445911088
Challenger outperforms Champion (or no Champion exists). Promoting...

## 3回
Champion model exists in Production
Challenger RMSE: 1.0672939052826609
Champion RMSE: 0.8180663645429364
Challenger did not outperform Champion. No promotion.